# This notebook runs a covariance based Monte Carlo propagation in ADAM

In [1]:
from adam import Batch
from adam import Batches
from adam import BatchRunManager
from adam import PropagationParams
from adam import OpmParams
from adam import ConfigManager
from adam import Projects
from adam import RestRequests
from adam import AuthenticatingRestProxy
from adam import AdamProcessingService
from adam import BatchPropagationResults
import time
import os

This sets up authenticated access to the server. It needs to be done before pretty much everything you want to do with ADAM.

In [2]:
# ConfigManager loads the config set up via adamctl.
# See the README at https://github.com/B612-Asteroid-Institute/adam_home/blob/master/README.md
config_manager = ConfigManager()
config_manager.set_default_env('dev')
config = config_manager.get_config()
auth_rest = AuthenticatingRestProxy(RestRequests())
aps = AdamProcessingService(auth_rest)

## Example Inputs

The PropagationParameters sent to the propagation API will operate on either Keplerian or Cartesian elements.

In [3]:
# from https://newton.spacedys.com/neodys/index.php?pc=1.1.1.1&n=2000SG344
# true_anomaly_deg is also available instead of mean_anomaly_deg
keplerian_elements = {
    'semi_major_axis_km': 146222900,
    'eccentricity': 0.066958,
    'inclination_deg': 0.112,
    'ra_of_asc_node_deg' : 191.912,
    'arg_of_pericenter_deg' :  275.347,
    'mean_anomaly_deg' : 35.681,
    'gm' : 132712440041.93938 #JPL Horizons GM
}

# Lower triangular covariance matrix (21 elements in a list)
covariance = [3.94346903514E+03, + \
             -1.40266786788E-04, 5.00812620000E-12, + \
             -2.91357694324E-04, 1.06017205000E-11, 3.15658331000E-11, + \
             -3.83826656095E-03, 1.40431472000E-10, 2.32155752000E-09, 8.81161492000E-07, + \
             -1.09220523817E-02, 3.62452521000E-10, -1.53067748000E-09, -8.70304198000E-07, 9.42413982000E-07, + \
             -2.96713683611E-01, 1.05830167000E-08, 2.23110293000E-08, 2.93564832000E-07, 7.81029359000E-07, 2.23721205000E-05]

### Set Parameters

Commented parameters are optional. Uncomment to use.

In [4]:
propagation_params = PropagationParams({
    'start_time': '2017-10-04T00:00:00Z',   # propagation start time in ISO format
    'end_time': '2017-10-11T00:00:00Z',     # propagation end time in ISO format

    'project_uuid': config['workspace'],
    'description': 'Jupyter Keplerian Covariance Monte Carlo Example',
    'monteCarloDraws': 10,
    'propagationType': 'MONTE_CARLO',
    'stopOnImpact': True,
    'step_size': 86400,
    'stopOnCloseApproach': False,
    'stopOnImpactDistanceMeters': 500000,
    'closeApproachRadiusFromTargetMeters': 7000000000
})

opm_params = OpmParams({
    'epoch': '2017-10-04T00:00:00Z',
    'keplerian_elements': keplerian_elements,
    'keplerian_covariance': covariance,   # object covariance
            
#     'mass': 500.5,              # object mass
#     'solar_rad_area': 25.2,     # object solar radiation area (m^2)
#     'solar_rad_coeff': 1.2,     # object solar radiation coefficient
#     'drag_area': 33.3,          # object drag area (m^2)
#     'drag_coeff': 2.5,          # object drag coefficient
    
})

### Submit and Run Propagation

In [5]:
batch_run = aps.execute_batch_propagation(config['workspace'], propagation_params, opm_params)
# batch_run = BatchPropagationResults.fromRESTwithRawIds(auth_rest, config['workspace'], 'efaa4ecc-c6d0-4d57-a769-cd400a3a0997')
print(batch_run)

{'job_uuid': '8bba9eec-7523-41e5-8af6-e610396ce1bf', 'results': None}


### Get Status

In [7]:
print(batch_run.check_status())
batch_run.wait_for_complete(print_waiting = True)
print()
print(batch_run.check_status())

COMPLETED

COMPLETED


### Get Number of Runs

In [8]:
runs_count = batch_run.get_result_ephemeris_count()
print(f'# Runs: {runs_count}')

# Runs: 11


### Get Summary Statistics

In [9]:
stats = batch_run.get_summary()
print(stats)

{'misses': 11, 'close_approach': 0, 'impacts': 0, 'pc': 0.0}


### Get Ephemeris for specific run

In [10]:
eph = batch_run.get_result_ephemeris(2)
print(eph)

       Epoch             X             Y             Z         Vx         Vy  \
0 2017-10-04 -1.171198e+08  6.777254e+07 -2.958866e+07 -18.050997 -23.957429   
1 2017-10-05 -1.186575e+08  6.569012e+07 -2.868176e+07 -17.542938 -24.245020   
2 2017-10-06 -1.201511e+08  6.358328e+07 -2.776420e+07 -17.029637 -24.522886   
3 2017-10-07 -1.216001e+08  6.145285e+07 -2.683634e+07 -16.511341 -24.790957   
4 2017-10-08 -1.230041e+08  5.929969e+07 -2.589855e+07 -15.988298 -25.049167   
5 2017-10-09 -1.243627e+08  5.712464e+07 -2.495120e+07 -15.460755 -25.297460   
6 2017-10-10 -1.256756e+08  5.492857e+07 -2.399467e+07 -14.928961 -25.535783   
7 2017-10-11 -1.269423e+08  5.271235e+07 -2.302933e+07 -14.393162 -25.764090   

          Vz  
0  10.433353  
1  10.558921  
2  10.680252  
3  10.797318  
4  10.910088  
5  11.018538  
6  11.122645  
7  11.222388  


### Get ending state vector

In [11]:
end_state_vectors = batch_run.get_final_positions(BatchPropagationResults.PositionOrbitType.CLOSE_APPROACH)
print("Close State Vectors")
if len(end_state_vectors) > 0:
    print(f'First close end state: {end_state_vectors[0]}')
else:
    print("None")


end_state_vectors = batch_run.get_final_positions(BatchPropagationResults.PositionOrbitType.IMPACT)
print("Impact State Vectors")
if len(end_state_vectors) > 0:
    print(f'First impact end state: {end_state_vectors[0]}')
else:
    print("None")


end_state_vectors = batch_run.get_final_positions(BatchPropagationResults.PositionOrbitType.MISS)
print("Miss State Vectors")
if len(end_state_vectors) > 0:
    print(f'First miss end state: {end_state_vectors[0]}')
else:
    print("None")



Close State Vectors
None
Impact State Vectors
None
Miss State Vectors
First miss end state: [-269287324584.73828, 16013894239.062584, -18148304071.962654]
